<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/120_%E6%A7%8B%E9%80%A0%E7%9A%84%E3%83%91%E3%82%BF%E3%83%BC%E3%83%B3%E3%83%9E%E3%83%83%E3%83%81%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

構造的パターンマッチング
========================

ソフトキーワード
----------------

**ソフトキーワード**（soft keywords）は、Python 3.10 から正式に導入された仕組みで、ある特定の文脈においてだけ予約されるキーワードである。他の場面ではそれらはキーワードとしては識別されず、普通に変数などの名前に使うことができる。つまり、ソフトキーワードは、字句解析の時点ではキーワードとしては識別されず、その後の構文解析において特定の構文でキーワードとして扱われる。

同じく Python 3.10 から導入された構造的パターンマッチングにおいて、`match`, `case`, `_` がソフトキーワードとされ、これらの名前を使っている既存のコードの互換性が保たれるようにしている。

なお、Python のソフトキーワードは、`keyword.softkwlist` で確認できる。

In [ ]:
keyword.softkwlist

['_', 'case', 'match']

構造的パターンマッチングの構文
------------------------------

**構造的パターンマッチング**（structural pattern matching）の構文は、次の通り:

``` python
match 式（または式のリスト）:
    case パターン[ if 条件]:
        処理
    ⋮
    case パターン[ if 条件]:
        処理
```

パターンは独自の構文に沿って記述されるものなので、<font color="red">パターンは式ではない</font>。

オプションの `if 条件` は**ガード**（guard）と呼ばれる。ガードの `条件` には、任意の式やセイウチ演算子 `:=` を書くことができる。

match 文の実行はおおむね以下のように進行する。

  1. `match` の右の式を評価して値を得る。この値を以下では「対象値」と呼ぶことにする。`match` の右にカンマを含む式のリストがある場合は、通常のルールに従ってタプル評価が行われる。
  2. 各パターンに対して、対象値がマッチするかどうかを（上から順番に）チェックする。
      * マッチングのチェックにより、パターン内の名前の一部あるいはすべてに対象値が束縛される（つまり代入が行われる）。具体的な束縛ルールはパターンの種類によって異なる。
  3. パターンに対象値がマッチした場合、該当のガードが（もしあれば）評価される。この場合、パターン内の名前がすべて束縛されていることが保証されている。
      * ガードの評価値が真であるか、もしくはガードがなければ、その case ブロックに記述された処理が実行され、match 文が終了する（次の case ブロックがあっても無視される）。
      * そうでなければ、次の case ブロックのパターンチェックに進む。
      * これ以上 case ブロックが存在しない場合は、match 文が終了する。

パターンは式ではないのに対して、ガードは式であるため副作用を起こすことができる。ガードがある case ブロックでは、先にパターンがチェックされ、そこでマッチしなかった場合は、ガードの評価は省略される。

他の制御構造（if 文など）と同様に、match 文のブロックでは独自のスコープが形成されない。**マッチが成功したパターンの中で束縛された名前は、そのパターンのブロック内だけでなく、match 文の後でも使用することができる**。

パターン
--------

以下の 10 種類のパターンは、構造的パターンマッチングで使えるパターンである。また、それらを組み合わせたものも、構造的パターンマッチングで使えるパターンである。

パターンには**論駁（ろんばく）不可能**（irrefutable）と**論駁可能**（refutable）の 2 つの形態がある。あらゆる値に合致するパターンは、論駁不可能であるという。**match 文の中で、論駁不可能な case ブロックは最大 1 つまで、かつ最後に位置する必要がある**。そうでない場合は、構文エラーが発生する。

### リテラルパターン ###

Python のリテラル（文字列、バイト列、整数、浮動小数点数、複素数）とシングルトンのうちの `None`、`True`、`False` は**リテラルパターン**である。ただし、f-strings はリテラルパターンから除外される（置換フィールドの評価が必要なため）。パターンは式ではないので、`100 + 200` や `max([1, 20, 0.3])` のように演算子や関数を使用することはできない。例外として、虚数リテラルに対する `+` や `-` は複素数のリテラルパターンに使える（例: `3 + 4j`）。

リテラルパターンは、`対象値 == リテラル` であるときマッチする。シングルトンである `None` と `True`、`False` は `is` 演算子を使って比較される。

リテラルパターンでは代入は行われない。

In [ ]:
text = "How are you?"

match text:
    case "How are you?":
        print("I'm great.")
    case "Nice to meet you.":
        print("Same here.")

I'm great.


### キャプチャパターン ###

変数は**キャプチャパターン**である。キャプチャパターンは論駁不可能であり、また、対象値が変数に代入される。

In [ ]:
num = 3

match num:
    case x:
        print(f"{num=}, {x=}")

assert x == 3  # 変数 x は match 文の後でも使える

num=3, x=3


### ワイルドカードパターン ###

文字 `'_'` は**ワイルドカードパターン**である。`'_'` は、`case` の後ろではあらゆる対象値がマッチするためのキーワードであって、変数ではない（ソフトキーワード）。つまり、ワイルドカードパターンは論駁不可能である。`'_'` は変数ではないから、このパターンでは代入は行われない。

In [ ]:
_ = "spam"  # _ をグローバル変数として使用
text = "ham"

match text:
    case "eggs":
        print("eggs!")
    case _:  # この _ は変数ではなくワイルドカードパターンを表すソフトキーワード
        print(f'{text=}, {_=}')  # この _ はグローバル変数を参照している

assert _ == "spam"

text='ham', _='spam'


### バリューパターン ###

ドット `.` 付き名前は**バリューパターン**である。このパターンは、指定された名前の値が対象値と等しい（比較演算子 `==` に基づく）ときにマッチする。このパターンでは代入は行われない。

パターンにドット `.` が付かない名前を使うと、キャプチャパターンになる。したがって、定数のマッチングを行いたい場合は、定数をクラス属性にしておいてバリューパターンを使うことになる。

In [ ]:
class Flags:
    flag1 = 1
    flag2 = 2

match 2:
    case Flags.flag1:
        print("flag1")
    case Flags.flag2:
        print("flag2")

flag2


上記のコードをクラスを使わないで次のように書く場合、どの case のパターンもキャプチャパターンであり、論駁不可能な case ブロックが最後でない場所にあるから構文エラーが発生する。

``` python
>>> FLAG1 = 1
>>> FLAG2 = 2
>>> match 2:
...     case FLAG1:  # この位置のキャプチャパターンは許されない
...         print("flag1")
...     case FLAG2:
...         print("flag2")
...
  File "<stdin>", line 2
SyntaxError: name capture 'FLAG1' makes remaining patterns unreachable
```

### シーケンスパターン ###

**シーケンスパターン**は、シーケンスに対して、その要素でマッチするためのパターンであり、1 つのサブパターン、または、カンマ区切りのサブパターンのリストを `[]` または `()` で括ったものである。2 種類の括弧に本質的な違いはない。このパターンで代入が行われるかどうかは、中のサブパターンによって決まる。

サブパターンは 1 つだけスター `*` を先頭に付けることができる（以下、スター付きサブパターンと呼ぶことにする）。スター付きサブパターンを含まないシーケンスパターンは固定長のシーケンスパターンであり、1 つのスター付きサブパターンを含むシーケンスパターンは可変長のシーケンスパターンである。

対象値がシーケンスパターンにマッチするかどうかは、以下のルールによって決められる。

**【固定長・可変長に共通するルール】**

  1. 対象値がシーケンスでない場合、マッチは失敗する。
  2. 対象値が文字列またはバイト列である場合、マッチは失敗する。

共通ルールによって、対象値はシーケンスであることが保証されるので、以下では対象シーケンスと呼ぶことにする。

**【シーケンスパターンが固定長の場合のルール】**

  1. 対象シーケンスの長さがサブパターンの数と等しくない場合、マッチは失敗する。
  2. シーケンスパターン内のサブパターンと、対象シーケンス内の対応する項目が左から右にチェックされる。サブパターンのマッチが失敗するとすぐに全体のチェックが打ち切られ、シーケンスパターンのマッチは失敗する。すべてのサブパターンが対応する項目とのマッチに成功すると、シーケンスパターンのマッチは成功する。

In [ ]:
def f(point):
    match point:
        case (0, 0):  # 2つのリテラルパターンを含むシーケンスパターン
            print("Origin")
        case (0, y):  # 1つのリテラルパターンと1つのキャプチャパターンを含むシーケンスパターン
            print(f"Y={y}")
        case (x, 0):  # 1つのキャプチャパターンと1つのリテラルパターンを含むシーケンスパターン
            print(f"X={x}")
        case (x, y):  # 2つのキャプチャパターンを含むシーケンスパターン
            print(f"X={x}, Y={y}")
        case _:
            raise ValueError("Not a point")

if __name__ == "__main__":
    f((0, 0))
    f((0, 2))
    f((1, 0))
    f((1, 2))

Origin
Y=2
X=1
X=1, Y=2


このコードでは、4 番目のシーケンスパターンは 2 個のキャプチャパターンだけを含むのであるが、論駁可能であることに注意する。対象値はシーケンスであるとは限らないからである。したがって、下にワイルドカードパターンがあっても構文エラーにはならない。

**【シーケンスパターンが可変長の場合のルール】**

  1. 対象シーケンスの長さがスターなしサブパターンの数より短い場合、マッチは失敗する。
  2. スターなしサブパターンおよびスター付きサブパターンと、対象シーケンス内の項目との対応は、『スター付き変数を含む変数リストへの代入』の場合に類似した対応となる。対応する項目とのマッチに成功すると、シーケンスパターンのマッチは成功する。そうでなければ、マッチは失敗する。

In [ ]:
def f(command):
    match command.split():
        case ["quit"]:
            print("Goodbye!")
        case ["go", direction]:
            print(f"Went to the {direction}")
        case ["drop", *objects]:
            for obj in objects:
                print(f"Dropped {obj}")
        case _:
            print(f"Sorry, I couldn't understand {command!r}")

if __name__ == "__main__":
    f("quit")
    f("go right")
    f("drop spam ham eggs")
    f("get knife")

Goodbye!
Went to the right
Dropped spam
Dropped ham
Dropped eggs
Sorry, I couldn't understand 'get knife'


シーケンスパターン内のサブパターンはワイルドカードパターンであってもよく、スターを付けてもよい（`*_`）。もちろん、シーケンスパターンは、ワイルドカードパターンやスター付きワイルドカードパターンを含んでいても、論駁可能である。

In [ ]:
def f(command):
    match command.split():
        case ["go", direction] if direction in ("right", "left"):
            print(f"Went to {direction}")
        case ["go", _]:
            print("Sorry, you can't go that way")

if __name__ == "__main__":
    f("go right")
    f("go back")

Went to right
Sorry, you can't go that way


### マッピングパターン ###

**マッピングパターン**は、マッピングオブジェクトに対して、キーと値でマッチするためのパターンであり、`{}` と `:` を使い、辞書と同様の形式をとり、キーと値をサブパターンで書いたものである。ただし、キーについては、次の制約がある。

  * キーは、リテラルパターンかバリューパターンでなければならない。
  * キーの重複は許可されない。すなわち
      * リテラルパターンのキーが重複すると、構文エラーが発生する。
      * 2 つのバリューパターンのキーが同じ値を持つ場合、実行時に `ValueError` 例外が発生する。

このパターンで代入が行われるかどうかは、値として書いたサブパターンによって決まる。

また、マッピングパターンは、可変長キーワード引数のように、最後のサブパターンだけ先頭に `**` が付けた形にできる。しかし、ワイルドカードパターンに `**` を付けた形 `**_` は冗長なため禁止される。

対象値がマッピングパターンにマッチするかどうかは、以下のルールによって決められる。

  1. 対象値がマッピングではない場合、マッチは失敗する。このルールにより、対象値はマッピングであることが保証されるので、以下では対象マッピングと呼ぶことにする。
  2. マッピングパターンのすべてのキーと値の組で、対象マッピングのキーと値の組をチェックし（マッピングパターン `{KEY1: P1, ...}` に対し `KEY1 in 対象値` が真で、`P1` が `対象値[KEY1]` にマッチするかというテストが行われる）、すべてマッチするする場合、（たとえマッピングパターンに存在しないキーを対象マッピングが持っていたとしても）マッピングパターンのマッチは成功する。
  3. マッピングパターンで重複キーが検出された場合、そのパターンは無効であるとみなされる。

In [ ]:
def f(data):
    match data:
        case {"氏名": name, "身長": height, "出身": come_from}:
            print(f"{name}さんの出身地は{come_from}")
        case {"氏名": name, "身長": height}:
            print(f"{name}さんの身長は{height}m")
        case {"氏名": name}:
            print(f"{name}さんのデータはありません")

if __name__ == "__main__":
    f({"氏名": "山田花子", "身長": 168})
    f({"氏名": "山田太郎", "出身": "鹿児島県", "身長": 170})

山田花子さんの身長は168m
山田太郎さんの出身地は鹿児島県


このコードで、もし最後の case ブロックが 1 番目だったなら、`'氏名'` キーをもつ対象値がすべてマッチするので、他の case ブロックはチェックされないことになる。また、もし 2 番目の case ブロックが 1 番目だったなら、`'氏名'` キーと `'身長'` キーをもつ対象値がすべてマッチするので、他のキーがつねに無視されることになる。

このように、2 つのマッピングパターンの case ブロックの間で、一方が他方のキーを全て含むような場合は、キーの数が少ないほうの case ブロックを上の位置にしてしまうと、キーの数が多いほうの case ブロックは無意味になってしまうので注意する。

### クラスパターン ###

**クラスパターン**は、オブジェクトに対して、そのクラスと属性でマッチするためのパターンであり、コンストラクター呼び出しの形式 `CLASS(KEY=PATTERN, ...)` をとるものである。`CLASS` でクラスを、キーワード引数の形 `KEY=PATTERN` で属性名と属性値を指定する。以下の制限がある。

  * キーワード引数の形 `KEY=PATTERN` では、`KEY` も `PATTERN` もパターンでなければならない。
  * 同じキーワードをクラスパターン内で繰り返してはならない。
  * 位置引数の形 `PATTERN` を使うこともできるが、パターンのチェック時にキーワード引数の形 `KEY=PATTERN` に暗黙的に変換される。この変換には、`CLASS` クラスの `__match_args__` 属性が使われる。この属性が定義されていない場合は、変換が失敗する。

このパターンで代入が行われるかどうかは、引数としたサブパターンによって決まる。

対象値がクラスパターンにマッチするかどうかは、以下のルールによって決められる。

  1. クラスパターン内のクラスが組み込み `type` のインスタンスではない場合、そのパターンは無効であるとみなされる（`TypeError` 例外が発生する）。
  2. 対象値がクラスパターン内のクラスのインスタンスではない場合（`isinstance()` でテスト）、マッチは失敗する。
  3. パターンの引数が存在しない場合、マッチは成功する。
  4. キーワード引数 `KEY=PATTERN` については、`hasattr(対象値, KEY)` が `True` で、かつ、`PATTERN` が `対象値.KEY` にマッチするかどうかがチェックされる。
  5. `i` 番目（0 から始まる）の位置引数 `PATTERN` は、クラスの `__match_args__` 属性により、`__match_args__[i]` で得られる文字列 `'KEY'` を使ってキーワード引数 `KEY=PATTERN` に変換される。たとえば、もし `MyClass.__match_args__` に `("left", "center", "right")` が定義されていた場合、 `case MyClass(x, y)` は `case MyClass(left=x, center=y)` と同義になる。パターンの引数の数は、`__match_args__` の要素数と同等かそれ以下でなければならない。重複するキーワードがあってはならない。すべての位置引数がキーワード引数に変換されたら、キーワード引数のみがあるかのようにチェックが進行する。変換に失敗したら、そのパターンは無効であるとみなされる（`TypeError` 例外が発生する）。
  6. `str` や `int` などの組み込みクラスに関するクラスパターンは単一の位置引数を受け入れ、そこにあるパターンが属性ではなく**オブジェクト全体とチェックされる**。

次のコードは、パターン引数が存在しないクラスパターンを使った例である:

In [ ]:
from decimal import Decimal

def f(x):
    match x:
        case complex():
            print(f"{x}は複素数")
        case Decimal():
            print(f"{x!r}は10進数")
        case float():
            print(f"{x}は浮動小数点数")
        case int():
            print(f"{x}は整数")

if __name__ == "__main__":
    f(10)
    f(1 + 2j)
    f(3.14)
    f(Decimal("3.14"))

10は整数
(1+2j)は複素数
3.14は浮動小数点数
Decimal('3.14')は10進数


次のコードは、キーワード引数のあるクラスパターンを使った例である:

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


def f(point):
    match point:
        case Point(x=0, y=0):
            print("Origin")
        case Point(x=0, y=y):
            print(f"Y={y}")
        case Point(x=x, y=0):
            print(f"X={x}")
        case Point():
            print("Somewhere else")
        case _:
            print("Not a point")


if __name__ == "__main__":
    f(Point(0, 0))
    f(Point(0, 1))
    f(Point(1, 0))
    f(Point(1, 1))
    f("Point")

Origin
Y=1
X=1
Somewhere else
Not a point


次のコードは、クラスに `__match_args__` 属性を定義して、位置引数のあるクラスパターンを使った例である:

In [ ]:
class Point:
    __match_args__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y


def f(point):
    match point:
        case Point(x, y) if x == y:
            print(f"Y=X at {x}")
        case Point(x, y):
            print("Not on the diagonal")


if __name__ == "__main__":
    f(Point(1, 1))
    f(Point(2, 1))

Y=X at 1
Not on the diagonal


このコードにおいて、クラスパターンの引数に、属性名を省略してキャプチャパターンが使用できていることに注目する。`__match_args__` 属性に基づいて、`case Point(x, y)` は `case Point('x'=x, 'y'=y)` に暗黙的に変換されてから `point` とマッチするか調べられる。もし `__match_args__` 属性がなければ、クラスパターンは無効とみなされる（`TypeError` 例外が発生する）。

次のコードは、マッピングパターンとクラスパターンを組み合わせたパターンの例である:

In [ ]:
def f(data):
    match data:
        case {"id": int(id)}:
            print(f"{id=} は古い形式のIDです")
        case {"id": str(id)}:
            print(f"{id=} は新しい形式のIDです")

if __name__ == "__main__":
    f({"id": 4492, "name": "山田太郎"})
    f({"id": "qA9s71p", "name": "山田花子"})

id=4492 は古い形式のIDです
id='qA9s71p' は新しい形式のIDです


このコードから、`int` や `str` などの組み込みクラスに関しては、クラスパターンの単一の引数（キャプチャーパターン `id`）に、属性値ではなくインスタンスそのものが束縛されていることがわかる。

### OR パターン ###

縦線 `|` で区切られた複数のパターンは、**OR パターン**である。ただし、以下の制限がある。

  1. 最後のサブパターン以外、論駁不可能であってはならない。
  2. 各サブパターンが束縛する名前の組み合わせは、すべて同じである必要がある。

OR パターンでは、対象値に対して順に各サブパターンのマッチングが行われる。マッチが成功するとそこで終了し、この OR パターンのマッチは成功したとみなされる。一方、どのサブパターンも成功しなければ、この OR パターンのマッチは失敗したことになる。**最後のサブパターンが論駁不可能である OR パターンは論駁不可能である**。

In [ ]:
def f(command):
    match command.split():
        case ["north"] | ["go", "north"]:
            print("Went north")
        case ["get", obj] | ["pick", "up", obj] | ["pick", obj, "up"]:
            print(f"Got {obj}")

if __name__ == "__main__":
    f("go north")
    f("pick ITEM up")

Went north
Got ITEM


このコードで、2 番目の OR パターンに、最後のサブパターンとしてキャプチャパターンを追加すると、論駁不可能となる。

In [ ]:
def f(command):
    match command.split():
        case ["north"] | ["go", "north"]:
            print("Went north")
        case ["get", obj] | ["pick", "up", obj] | ["pick", obj, "up"] | obj:
            print(obj)

if __name__ == "__main__":
    f("fall back")

['fall', 'back']


### AS パターン ###

`パターン as キャプチャパターン` の形は **AS パターン**である。

`as` の前のパターンが対象値にマッチしなければ、この AS パターンのマッチは失敗となる。`as` の前のパターンが対象値にマッチすれば、この AS パターンのマッチは成功となり、対象値がキャプチャパターンの変数に代入される。したがって、`as` の前のパターンが論駁不可能の場合、この AS パターンは論駁不可能となる。

In [ ]:
class Person:
    def __init__(self, name, relationship):
        self.name = name
        self.relationship = relationship

def f(person):
    match person:
        case Person(name="山田太郎") | Person(name="山田花子") as p:
            print(f"{p.name}は{p.relationship}です")
        case _:
            print("知らない人です")

if __name__ == "__main__":
    f(Person("山田太郎", "友人"))
    f(Person("山田花子", "いとこ"))

山田太郎は友人です
山田花子はいとこです


### グループパターン ###

パターンの周囲に括弧 `()` を追加することができる。この括弧は単に可読性を高める目的で使われるだけであり、構文上の意味が付与されるわけではない。

In [ ]:
def f(command):
    match command.split():
        case ["go", ("north" | "south" | "east" | "west") as direction]:
            print(f"Went {direction}")

if __name__ == "__main__":
    f("go west")

Went west
